---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    pd.set_option('display.max_rows',600)
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    untw = open('university_towns.txt')
    df = pd.DataFrame()
    for line in untw:
        #print(line)
        if re.search('\[edit].*', line):
            pos = re.search('\[.*', line).start()
            st = line[:pos].rstrip()
            continue
        else:
            if re.search('\(.*', line):
                pos = re.search('\(.*', line).start()
                region = line[:pos].rstrip()
            else:
                region = line.rstrip()
        
        df = df.append(pd.Series({'State':st,'RegionName':region}),ignore_index=True)
    return df.reindex(columns = ['State','RegionName'])
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    rec_start =pd.read_excel('gdplev.xls',skiprows = 5,usecols=[4,6])
    rec_start = rec_start.iloc[214:].rename(
        columns={'Unnamed: 4':'Y_Q',
                 'GDP in billions of chained 2009 dollars.1':'GDP'}).set_index('Y_Q')
    rec_start['Previous'] = rec_start['GDP'].shift(1)
    rec_start['Next'] = rec_start['GDP'].shift(-1)
    return rec_start[(rec_start['GDP'] < rec_start['Previous']) & (rec_start['GDP'] > rec_start['Next']) ].index[0]
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    rec_end =pd.read_excel('gdplev.xls',skiprows = 5,usecols=[4,6])
    rec_end = rec_end.iloc[214:].rename(
        columns={'Unnamed: 4':'Y_Q',
                 'GDP in billions of chained 2009 dollars.1':'GDP'}).set_index('Y_Q')
    rec_end['Previous'] = rec_end['GDP'].shift(1)
    rec_end['Next'] = rec_end['GDP'].shift(-1)
    return rec_end[(rec_end['GDP'] > rec_end['Previous']) & (rec_end['GDP'] < rec_end['Next']) & (rec_end.index >= get_recession_start())].index[1]
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    rec =pd.read_excel('gdplev.xls',skiprows = 5,usecols=[4,6])
    rec = rec.iloc[214:].rename(
        columns={'Unnamed: 4':'Y_Q',
                 'GDP in billions of chained 2009 dollars.1':'GDP'}).set_index('Y_Q')
    rec_start = get_recession_start()
    rec_end = get_recession_end()
    rec = rec.loc[rec_start:rec_end]
    return rec['GDP'].idxmin()
get_recession_bottom()

'2009q2'

In [7]:
print(str(2016)+ 'q1')

2016q1


In [8]:
pd.set_option('display.max_rows',1000)

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean values in a dataframe.
    This dataframe should be a dataframe with columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    hdata = pd.read_csv('City_Zhvi_AllHomes.csv')
    hdata = hdata.loc[:,hdata.columns.difference(hdata.columns[ range(6,63) ])] #keeps only cols from 2001-01 to 2016-08
    for key in states.keys():
        hdata['State'] = hdata['State'].replace(key,states[key])                #replace names in index with values in dictionary
    
    cols = list(hdata.columns[ range(0,188) ])                                  # get list of all year and quarte name in hdata.columns
    hdata = hdata.set_index(['State','RegionName'])
    
    nonReqdf = hdata.iloc[:,188:].sort_index()
    hdata = hdata.drop(['CountyName','Metro','RegionID','SizeRank'],axis=1)
    
    hdata.columns = pd.to_datetime(hdata.columns)
    hdata = hdata.resample('Q',axis=1).mean()
    hdata = hdata.rename(columns=lambda x: str(x.to_period('Q')).lower()).sort_index()
    hdata = pd.concat([hdata,nonReqdf],axis=1)

    return hdata

convert_housing_data_to_quarters()#.loc[[('Ohio','Akron'),('Ohio','Dayton')]].loc[:,['2010q3','2015q2','2016q4']]

2001q1         2001q2         2001q3  \
State   RegionName                                                 
Alabama Adamsville    73466.666667   74000.000000   73333.333333   
        Alabaster    123233.333333  125133.333333  127766.666667   
        Albertville   75900.000000   76000.000000   72066.666667   
        Arab          84200.000000   84533.333333   81666.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        115833.333333  117200.000000  117800.000000   
        Casper        95500.000000   97633.333333   99433.333333   
        Cheyenne     125533.333333  126300.000000  126466.666667   
        Evansville   130566.666667  131433.333333  132400.000000   
        Pine Bluffs  103233.333333  104600.000000  106500.000000   

                            2001q4         2002q1         2002q2  \
State   RegionName                                                 
Alabama Adamsville    73100.000000   73333.333333   73133.333333   
        Alabaster    127200.000000  127300.000000  128000.000000   
        Albertville   73566.666667   76533.333333   76366.666667   
        Arab          83900.000000   87266.666667   87700.000000   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117633.333333  117333.333333  117233.333333   
        Casper       100633.333333  101733.333333  101533.333333   
        Cheyenne     128133.333333  128466.666667  129633.333333   
        Evansville   133466.666667  133300.000000  131066.666667   
        Pine Bluffs  104066.666667  102233.333333  103566.666667   

                            2002q3         2002q4         2003q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   75733.333333   77366.666667   
        Alabaster    127666.666667  129100.000000  131166.666667   
        Albertville   74433.333333   73033.333333   74333.333333   
        Arab          88233.333333   85966.666667   86233.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        118366.666667  123833.333333  127433.333333   
        Casper       102400.000000  102566.666667  102933.333333   
        Cheyenne     131200.000000  134100.000000  138200.000000   
        Evansville   133300.000000  133833.333333  132600.000000   
        Pine Bluffs  105866.666667  110833.333333  113766.666667   

                            2003q2  ...         2015q2         2015q3  \
State   RegionName                  ...                                 
Alabama Adamsville    79233.333333  ...   73033.333333   73933.333333   
        Alabaster    134133.333333  ...  149566.666667  150366.666667   
        Albertville   75466.666667  ...   89500.000000   90233.333333   
        Arab          87366.666667  ...  109433.333333  110900.000000   
        Ardmore                NaN  ...  143000.000000  144600.000000   
...                            ...  ...            ...            ...   
Wyoming Burns        127133.333333  ...  163066.666667  164600.000000   
        Casper       106100.000000  ...  182633.333333  183300.000000   
        Cheyenne     140966.666667  ...  183266.666667  186766.666667   
        Evansville   139633.333333  ...  303166.666667  300966.666667   
        Pine Bluffs  108433.333333  ...  167433.333333  167466.666667   

                            2015q4         2016q1         2016q2    2016q3  \
State   RegionName                                                           
Alabama Adamsville    73866.666667   74166.666667   74933.333333   74700.0   
        Alabaster    151733.333333  153466.666667  155100.000000  155850.0   
        Albertville   91366.666667   92000.000000   92466.666667   92200.0   
        Arab         112233.33333

In [72]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    unv_town = get_list_of_university_towns()
    housing  = convert_housing_data_to_quarters()
    rec_start = get_recession_start()
    rec_bot  = get_recession_bottom()
    rec_end  = get_recession_end()
    
    unvtown = pd.merge(housing,unv_town,
                       how='inner',
                       left_index=True,
                       right_on=['State','RegionName']).set_index(['State','RegionName']).sort_index()
    #left outer join exclusion can be achieved by setting pandas merge's indicator=True.
    #Then filter by the indicator in _merge column.
    non_unvtown = pd.merge(housing,unv_town,how='left',indicator=True,left_index=True,right_on=['State','RegionName']).set_index(['State','RegionName']).sort_index()
    non_unvtown= non_unvtown[non_unvtown['_merge'] == 'left_only']
    
    #unvtown.columns.get_loc[rec_start] = 30
    unvtown = unvtown.iloc[:,29:36] 
    unvtown['Growth'] = unvtown[rec_start] - unvtown[rec_bot]
    unvtown['Price Ratio'] = np.float64(unvtown.iloc[:,0]) / np.float64(unvtown[rec_bot])
    
    non_unvtown = non_unvtown.iloc[:,29:36]
    non_unvtown['Growth'] = non_unvtown[rec_start] - non_unvtown[rec_bot] 
    non_unvtown['Price Ratio'] = np.float64(non_unvtown.iloc[:,0]) / np.float64(non_unvtown[rec_bot])
    res = ttest_ind(unvtown['Price Ratio'],non_unvtown['Price Ratio'],nan_policy='omit')
    if res[1]<0.01:
        different = True
        if unvtown['Price Ratio'].mean() < non_unvtown['Price Ratio'].mean():
            better = 'university town'
        else:
            better = 'non-university town'
    else:
        different = False
        if unvtown['Price Ratio'].mean() < non_unvtown['Price Ratio'].mean():
            better = 'university town'
        else:
            better = 'non-university town'
    return (different,res[1],better)
run_ttest()

(True, 0.002724063704753125, 'university town')